## Preprocessing

In [35]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [36]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"], axis=1)

In [37]:
# Determine the number of unique values in each column.
uniques = application_df.nunique()
print(uniques)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [38]:
# Look at APPLICATION_TYPE value counts for binning
typeCounts = application_df["APPLICATION_TYPE"].value_counts()
print(typeCounts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [39]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 250
application_types_to_replace = list(typeCounts[typeCounts < cutoff].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [40]:
# Look at CLASSIFICATION value counts for binning
classCounts = application_df["CLASSIFICATION"].value_counts()
print(classCounts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [41]:
# You may find it helpful to look at CLASSIFICATION value counts >1
counts = classCounts[classCounts > 1]
print(counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [42]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 500
classifications_to_replace = list(classCounts[classCounts < cutoff].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [43]:
# Convert categorical data to numeric with `pd.get_dummies`
columns = ["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT", "SPECIAL_CONSIDERATIONS"]
application_df = pd.get_dummies(application_df, columns=columns, dtype=int)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [44]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop("IS_SUCCESSFUL", axis=1).values
y = application_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [45]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

Sidequest to determine optimal nodes and layers

In [46]:
len(X_train[0])

44

In [47]:
dimensions = 44
binaryOutput = True
MAX_NEURONS = 100
MAX_LAYERS = 10

def create_model(hp):
    nnModel = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nnModel.add(tf.keras.layers.Dense(units=hp.Int('first_units', min_value=1, max_value=MAX_NEURONS, step=2), activation=activation, input_dim=dimensions))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, MAX_LAYERS)):
        nnModel.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i), min_value=1, max_value=MAX_NEURONS, step=2), activation=activation))

    # Last layer for binary classification
    if(binaryOutput):  
        nnModel.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nnModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return nnModel

In [49]:
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2
)

tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))
bestParm = tuner.get_best_hyperparameters(1)[0]
bestParm.values

Trial 60 Complete [00h 00m 23s]
val_accuracy: 0.7258309125900269

Best val_accuracy So Far: 0.7280466556549072
Total elapsed time: 00h 12m 01s


{'activation': 'relu',
 'first_units': 85,
 'num_layers': 3,
 'units_0': 23,
 'units_1': 53,
 'units_2': 39,
 'units_3': 41,
 'units_4': 71,
 'units_5': 21,
 'units_6': 85,
 'units_7': 99,
 'units_8': 63,
 'units_9': 23,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [50]:
#This is a model that limits itself to whatever the "num_layers" attribute of the "tuner.get_best_hyperparameters(1)[0]" dictionary
def create_defined_model(inputDims, hyperparms):
    # Define the model
    model = tf.keras.models.Sequential()

    # Add the first hidden layer
    model.add(tf.keras.layers.Dense(units=hyperparms['first_units'], activation=hyperparms['activation'], input_dim=inputDims))

    # Add the specified number of hidden layers with specified units and activation
    for i in range(hyperparms['num_layers']):
        units_key = f'units_{i}'
        units = 5 if units_key not in hyperparms else hyperparms[units_key]
        activation = hyperparms['activation']
        model.add(tf.keras.layers.Dense(units=units, activation=activation))

    # Add the output layer
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [55]:
#This model uses all the layers that Keras Tuner provided (i.e. 6)
def forced_model(inputDims, hyperparms):
    # Define the model
    model = tf.keras.models.Sequential()

    # Add the first hidden layer
    model.add(tf.keras.layers.Dense(units=hyperparms['first_units'], activation=hyperparms['activation'], input_dim=inputDims))

    # Add hidden layer
    model.add(tf.keras.layers.Dense(units=hyperparms['units_0'], activation=hyperparms['activation']))

    # Add hidden layer
    model.add(tf.keras.layers.Dense(units=hyperparms['units_1'], activation=hyperparms['activation']))

    # Add hidden layer
    model.add(tf.keras.layers.Dense(units=hyperparms['units_2'], activation=hyperparms['activation']))

    # Add hidden layer
    model.add(tf.keras.layers.Dense(units=hyperparms['units_3'], activation=hyperparms['activation']))

    # Add hidden layer
    model.add(tf.keras.layers.Dense(units=hyperparms['units_4'], activation=hyperparms['activation']))

    # Add hidden layer
    model.add(tf.keras.layers.Dense(units=hyperparms['units_5'], activation=hyperparms['activation']))
    
    # Add hidden layer
    model.add(tf.keras.layers.Dense(units=hyperparms['units_6'], activation=hyperparms['activation']))
    
    # Add hidden layer
    model.add(tf.keras.layers.Dense(units=hyperparms['units_7'], activation=hyperparms['activation']))
    
    # Add hidden layer
    model.add(tf.keras.layers.Dense(units=hyperparms['units_8'], activation=hyperparms['activation']))
    
    # Add hidden layer
    model.add(tf.keras.layers.Dense(units=hyperparms['units_9'], activation=hyperparms['activation']))

    # Add the output layer
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [56]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = create_defined_model(44, bestParm)
#nn = forced_model(44, bestParm)
# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 85)                3825      
                                                                 
 dense_9 (Dense)             (None, 23)                1978      
                                                                 
 dense_10 (Dense)            (None, 53)                1272      
                                                                 
 dense_11 (Dense)            (None, 39)                2106      
                                                                 
 dense_12 (Dense)            (None, 41)                1640      
                                                                 
 dense_13 (Dense)            (None, 71)                2982      
                                                                 
 dense_14 (Dense)            (None, 21)               

In [18]:
# Compile the model (COMIPLED IN CONSTRUCTION)
#nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [57]:
# Train the model
from tensorflow.keras.callbacks import EarlyStopping

# Create an EarlyStopping callback
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

fit_model = nn.fit(X_train_scaled, y_train, epochs=500, callbacks=[early_stopping])

Epoch 1/500
804/804 [==============================] - 3s 2ms/step - loss: 0.5709 - accuracy: 0.7251
Epoch 2/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5558 - accuracy: 0.7290
Epoch 3/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5525 - accuracy: 0.7315
Epoch 4/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5505 - accuracy: 0.7308
Epoch 5/500
804/804 [==============================] - 1s 2ms/step - loss: 0.5503 - accuracy: 0.7313
Epoch 6/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7334
Epoch 7/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5476 - accuracy: 0.7349
Epoch 8/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5473 - accuracy: 0.7352
Epoch 9/500
804/804 [==============================] - 1s 2ms/step - loss: 0.5471 - accuracy: 0.7340
Epoch 10/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5457 - accura

In [58]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5579 - accuracy: 0.7229 - 431ms/epoch - 2ms/step
Loss: 0.5578835010528564, Accuracy: 0.7229154706001282


In [54]:
# Export our model to HDF5 file
#nn.save("AlphabetSoupCharity.h5")
#nn.save("AlphabetSoupCharity_Optimization.h5")
#nn.save("AlphabetSoupCharity_Optimization_alt.h5")
nn.save("AlphabetSoupCharity_Optimization_final.h5")

d:\Anaconda\envs\dev\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
